# Start

- This a first step into extracting the chapters font size for a more structured & enriched chunking. The chapter font size is saved in a json file and can be used in the future to extract the chapters more accurately. - `This notebook was used to detect the font size of the chapter titles in the PDF files. The font size is then saved in a JSON file for future use.`.

In [14]:
import fitz

if not hasattr(fitz.Page, "find_tables"):
    raise RuntimeError("This PyMuPDF version does not support the table feature")

# Functions

In [42]:
CURATED_PATH = r".\documents\curated"
import os
DOC_PATH = os.path.join(CURATED_PATH, "doc_001.pdf")
# DOC_PATH = os.path.join(CURATED_PATH, "doc_002.pdf")
# DOC_PATH = os.path.join(CURATED_PATH, "doc_003.pdf")
# DOC_PATH = os.path.join(CURATED_PATH, "doc_004.pdf")
# DOC_PATH = os.path.join(CURATED_PATH, "doc_005.pdf") # single chapter

In [43]:
# lets see the diffent police sizes in the document
def get_font_sizes(doc_path, page):
    doc = fitz.open(doc_path)
    page = doc.load_page(page)
    blocks = page.get_text("dict")["blocks"]
    font_sizes = set()
    for block in blocks:
        if block["type"] == 0:  # text block
            for line in block["lines"]:
                for span in line["spans"]:
                    font_sizes.add(span["size"])
    return sorted(font_sizes)

    

In [44]:
font_sizes = get_font_sizes(DOC_PATH, 0)
print(font_sizes)
#print exepmle of the text with the different font sizes
def get_text_by_font_size(doc_path, page, font_size):
    doc = fitz.open(doc_path)
    page = doc.load_page(page)
    blocks = page.get_text("dict")["blocks"]
    texts = []
    for block in blocks:
        if block["type"] == 0:  # text block
            for line in block["lines"]:
                for span in line["spans"]:
                    if span["size"] == font_size:
                        texts.append(span["text"])
    return texts
for size in font_sizes:
    texts = get_text_by_font_size(DOC_PATH, 0, size)
    print(f"Font size: {size}, Example text: {texts[:3]}")

[6.49656867980957, 8.662057876586914, 10.0, 17.819089889526367, 29.698484420776367, 46.71791458129883, 75.23616790771484]
Font size: 6.49656867980957, Example text: ['CHAPTER']
Font size: 8.662057876586914, Example text: [' ', ' ', ' 1    ']
Font size: 10.0, Example text: ['edical emergencies can happen every day, in any setting. People are injured in situations like falls or ', 'motor-vehicle accidents, or they develop sudden illnesses, such as heart attack or stroke.', 'The statistics are sobering. For example, about 900,000 people in the United States die each year from some ']
Font size: 17.819089889526367, Example text: ['CHAPTER     ']
Font size: 29.698484420776367, Example text: ['Before Giving Care and Checking ', 'an Injured or Ill Person']
Font size: 46.71791458129883, Example text: ['M']
Font size: 75.23616790771484, Example text: ['1']


In [45]:
CHAPTER_FONT_SIZE = 29

In [46]:
import os
import json

FONT_SIZE_FILE = "chapter_font_sizes.json"

def save_chapter_font_size(doc_path, font_size):
    # Extract document name safely
    doc_name = os.path.splitext(os.path.basename(doc_path))[0]

    new_entry = {
        "doc_path": doc_name,
        "chapter_font_size": font_size
    }

    # Load existing data if file exists
    if os.path.exists(FONT_SIZE_FILE):
        with open(FONT_SIZE_FILE, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                data = []
    else:
        data = []

    # Update if exists, otherwise append
    updated = False
    for entry in data:
        if entry["doc_path"] == doc_name:
            entry["chapter_font_size"] = font_size
            updated = True
            break

    if not updated:
        data.append(new_entry)

    # Save full valid JSON array
    with open(FONT_SIZE_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

    print(f"Chapter font size for {doc_name} saved.")


In [ ]:
# save_chapter_font_size(DOC_PATH, CHAPTER_FONT_SIZE)


Chapter font size for doc_001 saved.


# END